In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [53]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in range(df_train.columns.size):
    df_train.iloc[:,i] = le.fit_transform(df_train.iloc[:,i])
    df_test.iloc[:,i] = le.fit_transform(df_test.iloc[:,i])

In [54]:
shuffled = df_train[df_train['ACTION']==1].sample(frac=1)
train = np.array_split(shuffled,5)

In [55]:
for i in range(len(train)):
    train[i] = train[i].append(df_train[df_train['ACTION']==0])

In [56]:
x = []
y = []
for i in range(len(train)):
    x.append(train[i].drop('ACTION',axis=1))
    y.append(train[i]['ACTION'])

In [70]:
test_input = df_test.drop('id',axis=1)

In [88]:
from sklearn.model_selection import train_test_split                                                  # x_train = y_train = x_valid = y_valid = [0]*5
x_train1, x_valid1, y_train1, y_valid1 = train_test_split(x[0],y[0],test_size=0.2,random_state=1)     #x_train and related are lists (of size 5) of dataframes
x_train2, x_valid2, y_train2, y_valid2 = train_test_split(x[1],y[1],test_size=0.2,random_state=1)
x_train3, x_valid3, y_train3, y_valid3 = train_test_split(x[2],y[2],test_size=0.2,random_state=1)
x_train4, x_valid4, y_train4, y_valid4 = train_test_split(x[3],y[3],test_size=0.2,random_state=1)
x_train5, x_valid5, y_train5, y_valid5 = train_test_split(x[4],y[4],test_size=0.2,random_state=1)

In [90]:
#CATBOOST MODEL on x_train[0], y_train[0]
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

cat_features = list(range(x_train1.columns.size))                         #classified features on which our model will train

cat_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric="AUC",
    random_seed=1,
    cat_features = cat_features
)

cat_model.fit(x_train1,y_train1,eval_set=(x_valid1,y_valid1),use_best_model=True)

Learning rate set to 0.050233
0:	test: 0.7137605	best: 0.7137605 (0)	total: 38.6ms	remaining: 38.5s
1:	test: 0.7090734	best: 0.7137605 (0)	total: 55.9ms	remaining: 27.9s
2:	test: 0.7432847	best: 0.7432847 (2)	total: 87.5ms	remaining: 29.1s
3:	test: 0.7599234	best: 0.7599234 (3)	total: 118ms	remaining: 29.4s
4:	test: 0.7708885	best: 0.7708885 (4)	total: 141ms	remaining: 28.1s
5:	test: 0.7810305	best: 0.7810305 (5)	total: 179ms	remaining: 29.7s
6:	test: 0.7905667	best: 0.7905667 (6)	total: 207ms	remaining: 29.3s
7:	test: 0.8054490	best: 0.8054490 (7)	total: 263ms	remaining: 32.6s
8:	test: 0.8132517	best: 0.8132517 (8)	total: 300ms	remaining: 33s
9:	test: 0.8137313	best: 0.8137313 (9)	total: 324ms	remaining: 32s
10:	test: 0.8256633	best: 0.8256633 (10)	total: 397ms	remaining: 35.7s
11:	test: 0.8350189	best: 0.8350189 (11)	total: 441ms	remaining: 36.3s
12:	test: 0.8386555	best: 0.8386555 (12)	total: 478ms	remaining: 36.3s
13:	test: 0.8406304	best: 0.8406304 (13)	total: 521ms	remaining: 36.

In [91]:
cat_boost_score = roc_auc_score(y_valid1,cat_model.predict(x_valid1))
print(cat_boost_score)                                                          #Accuracy for cat_boost model

0.7945022989555737


In [107]:
#ADABOOST CLASSIFIER
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(learning_rate=1,n_estimators=10000,random_state=1)
clf.fit(x_train2,y_train2)

AdaBoostClassifier(learning_rate=1, n_estimators=10000, random_state=1)

In [108]:
clf_score = roc_auc_score(y_valid2,clf.predict(x_valid2))
print(clf_score)                                                                #Accuracy for adaboost model

0.6560462314093446


In [110]:
#XGBOOST 
import xgboost as xgb
model_xgb = xgb.XGBClassifier(learning_rate=1,n_estimators=100,eval_metric='auc')
eval_set = [(x_valid3,y_valid3)]
model_xgb.fit(x_train3,y_train3,eval_set=eval_set)

[0]	validation_0-auc:0.66387
[1]	validation_0-auc:0.72863
[2]	validation_0-auc:0.74588
[3]	validation_0-auc:0.76822
[4]	validation_0-auc:0.77525
[5]	validation_0-auc:0.78204
[6]	validation_0-auc:0.78598
[7]	validation_0-auc:0.79247
[8]	validation_0-auc:0.79468
[9]	validation_0-auc:0.79747
[10]	validation_0-auc:0.80144
[11]	validation_0-auc:0.80355
[12]	validation_0-auc:0.80005
[13]	validation_0-auc:0.80328
[14]	validation_0-auc:0.80827
[15]	validation_0-auc:0.80918
[16]	validation_0-auc:0.80848
[17]	validation_0-auc:0.81029
[18]	validation_0-auc:0.81057
[19]	validation_0-auc:0.81193
[20]	validation_0-auc:0.81114
[21]	validation_0-auc:0.81247
[22]	validation_0-auc:0.81691
[23]	validation_0-auc:0.81781
[24]	validation_0-auc:0.82203
[25]	validation_0-auc:0.82246
[26]	validation_0-auc:0.82436
[27]	validation_0-auc:0.82466
[28]	validation_0-auc:0.82372
[29]	validation_0-auc:0.82395
[30]	validation_0-auc:0.82258
[31]	validation_0-auc:0.82507
[32]	validation_0-auc:0.82415
[33]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [111]:
xgb_score = roc_auc_score(y_valid3,model_xgb.predict(x_valid3))
print(xgb_score)                                                        #Accuracy for xgboost model

0.76574343123465


In [112]:
#RANDOM FOREST CLASSIFIER
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(x_train4,y_train4)

RandomForestClassifier(n_estimators=1000)

In [113]:
rfc_score = roc_auc_score(y_valid4,rfc.predict(x_valid4))
print(rfc_score)                                                        #Accuracy for random forest

0.7636875917687054


In [114]:
#KNN 
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(weights='distance')
neigh.fit(x_train5,y_train5)

KNeighborsClassifier(weights='distance')

In [120]:
neigh_score = roc_auc_score(y_valid5,neigh.predict(x_valid5))
print(neigh_score)                                                      #Accuracy for knn model

0.6079464533737813


In [124]:
result_cat = cat_model.predict(test_input)
result_clf = clf.predict(test_input)
result_xgb = model_xgb.predict(test_input)
result_rfc = rfc.predict(test_input)
result_neigh = neigh.predict(test_input)

In [136]:
a = [2,3]
b = [4,5]
result = np.average(a,b)
result

TypeError: an integer is required (got type list)